# Flutter
* Professor: Elias Oliveira


# Persistência de dados Local.
* Vimos a importância de ter uma aplicação onde o estado(dados) se alteram, porem se a aplicação não persistir seus dados em um repositório, arquivo ou base de dados, não podemos interromper a execução do programa e retomar do ponto que paramos. 
Para isso devemos persistir os dados numa memória não volatil.
* Gravar as informações em arquivos é uma boa alternativa, se associarmos a isso um padrão independente de plataforma teremos nossa aplicação executando com perfeição em qualquer dispositivo.

### Ler e gravar arquivos
* Em alguns casos, você precisa ler e gravar arquivos no disco. Por exemplo, talvez seja necessário persistir os dados nas inicializações de aplicativos, ou baixar dados da internet e salvá-los para uso off-line posterior.

* Para salvar arquivos em disco em aplicativos móveis ou de desktop, Combine o plugin ***path_provider*** com a biblioteca ***Dart:IO***.


## Programação assincrona
* As operações assíncronas permitem que o programa conclua o trabalho enquanto aguarda o término de outra operação. Aqui estão algumas operações assíncronas comuns:
    - Buscar dados em uma rede.
    - Gravando em um banco de dados.
    - Lendo dados de um arquivo.

* Vamos usar dois exemplos de função sincrona e assincrona. 

In [ ]:
//Example: synchronous functions
String createOrderMessage() {
  var order = fetchUserOrder();
  return 'Your order is: $order';
}

Future<String> fetchUserOrder() =>
    // Imagine that this function is
    // more complex and slow.
    Future.delayed(
      const Duration(seconds: 2),
      () => 'Large Latte',
    );

void main() {
  print('Fetching user order...');
  print(createOrderMessage());
}

In [ ]:
//Example: asynchronous functions
Future<String> createOrderMessage() async {
  var order = await fetchUserOrder();
  return 'Your order is: $order';
}

Future<String> fetchUserOrder() =>
    // Imagine that this function is
    // more complex and slow.
    Future.delayed(
      const Duration(seconds: 2),
      () => 'Large Latte',
    );

Future<void> main() async {
  print('Fetching user order...');
  print(await createOrderMessage());
}

### 1. Encontre o caminho local correto

* Vamos salvar em um arquivo o estado do contador e iniciar a contagem a partir do último valor salvo antes de sairmos da aplicação
* Onde você deve armazenar esses dados?
    * O pacote path_provider fornece uma maneira independente de plataforma para acessar locais comumente usados no sistema de arquivos do dispositivo. O plugin atualmente suporta acesso a Dois locais de sistema de arquivos:
    Diretório temporário
        1. Um diretório temporário (cache) que o sistema pode limpar a qualquer momento. 
        2. Diretório de documentos. Um diretório para o aplicativo armazenar arquivos que somente ele pode acessar. O sistema limpa o diretório somente quando o aplicativo é eliminado.

In [ ]:
Future<String> get _localPath async {
  final directory = await getApplicationDocumentsDirectory();

  return directory.path;
}

### 2. Crie uma referência ao local do arquivo
* Depois de saber onde armazenar o arquivo, crie uma referência ao localização completa do arquivo. Você pode usar a classe File da biblioteca dart:io para conseguir isso.

In [ ]:
Future<File> get _localFile async {
  final path = await _localPath;
  return File('$path/counter.txt');
}

### 3. Gravar dados no arquivo
* O contador é um inteiro, mas é gravado no como uma cadeia de caracteres usando a sintaxe.File'$counter'

In [ ]:
Future<File> writeCounter(int counter) async {
  final file = await _localFile;

  // Write the file
  return file.writeAsString('$counter');
}

### 4. Leia os dados do arquivo
* Agora que você tem alguns dados no disco, você pode lê-los. Mais uma vez, use a classe.File

In [ ]:
Future<int> readCounter() async {
  try {
    final file = await _localFile;

    // Read the file
    final contents = await file.readAsString();

    return int.parse(contents);
  } catch (e) {
    // If encountering an error, return 0
    return 0;
  }
}

### Exemplo completo.

In [ ]:
Exemplo completo
import 'dart:async';
import 'dart:io';

import 'package:flutter/material.dart';
import 'package:path_provider/path_provider.dart';

void main() {
  runApp(
    MaterialApp(
      title: 'Reading and Writing Files',
      home: FlutterDemo(storage: CounterStorage()),
    ),
  );
}

class CounterStorage {
  Future<String> get _localPath async {
    final directory = await getApplicationDocumentsDirectory();

    return directory.path;
  }

  Future<File> get _localFile async {
    final path = await _localPath;
    return File('$path/counter.txt');
  }

  Future<int> readCounter() async {
    try {
      final file = await _localFile;

      // Read the file
      final contents = await file.readAsString();

      return int.parse(contents);
    } catch (e) {
      // If encountering an error, return 0
      return 0;
    }
  }

  Future<File> writeCounter(int counter) async {
    final file = await _localFile;

    // Write the file
    return file.writeAsString('$counter');
  }
}

class FlutterDemo extends StatefulWidget {
  const FlutterDemo({super.key, required this.storage});

  final CounterStorage storage;

  @override
  State<FlutterDemo> createState() => _FlutterDemoState();
}

class _FlutterDemoState extends State<FlutterDemo> {
  int _counter = 0;

  @override
  void initState() {
    super.initState();
    widget.storage.readCounter().then((value) {
      setState(() {
        _counter = value;
      });
    });
  }

  Future<File> _incrementCounter() {
    setState(() {
      _counter++;
    });

    // Write the variable as a string to the file.
    return widget.storage.writeCounter(_counter);
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Reading and Writing Files'),
      ),
      body: Center(
        child: Text(
          'Button tapped $_counter time${_counter == 1 ? '' : 's'}.',
        ),
      ),
      floatingActionButton: FloatingActionButton(
        onPressed: _incrementCounter,
        tooltip: 'Increment',
        child: const Icon(Icons.add),
      ),
    );
  }
}

### Desafio
* Inclua mais dois botões na aplicação para que salvar não seja automático:
    * salva contador e,
    * restaura contador

## Bibliografia
* Beginning Flutter®: A Hands On Guide To App Development, First Edition.Marco L. Napoli. Publicado em 2020 por John Wiley & Sons, Inc.
* https://docs.flutter.dev/cookbook/persistence/reading-writing-files
